In [1]:
# initiation
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

     |████████████████████████████████| 719 kB 23.9 MB/s 
     |████████████████████████████████| 365 kB 64.0 MB/s 
     |████████████████████████████████| 1.3 MB 58.5 MB/s 
     |████████████████████████████████| 4.7 MB 49.8 MB/s 
     |████████████████████████████████| 115 kB 73.1 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 212 kB 72.8 MB/s 
     |████████████████████████████████| 127 kB 66.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
Mounted at /content/gdrive


# Collaborative filtering 협업 필터링

광고, 넷플릭스 동영상 추천 등의 알고리즘은 협업 필터링으로부터 나온다. \
아이언맨을 좋아한 유저 A는 토르, 헐크에 대해서도 높은 평가를 제공했으며, \
유저 B 역시 아이언맨을 좋아하지만 다음 동영상을 보이게 망설이고 있다. 이때 컴퓨터는 유저 A의 평가를 참조하여\
토르와 헐크의 영화를 추천해준다. 이와 같은 방식이 협업 필터링이다.

컴퓨터는 항목과 관련된 정보를 가지고 있지 않다. \
그렇지만 항목을 선택한 이유와 그 항목이 가지는 의미를 표현하는 방법이 있어야 하며, 잠재 요소가 이를 잠재적으로 표현한다.

In [2]:
# 무비렌즈의 데이터셋을 들고온다. 평점 정보와 사용자 아이디, 평점 정보의 조합을 제공한다.

from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [3]:
# 열이름이 임베딩 되지 않았으므로 pandas를 통해 열 때 지정해주기

ratings = pd.read_csv(path/'u.data', delimiter = '\t', header=None,
                      names = ['user', 'movie', 'rating', 'timestamp'])
ratings.head()

# 각 컬럼별로 유저의 아이디, 영화 아이디, 평점, 타임스태프를 표현한다.

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
# -1 ~ +1까지 요솟값을 표현할 때 음수면 약한 매치, 양수면 강한 매치
# SF, Action, 고전영화를 나타내는 요소를 표현하면 다음과 같다
last_skywalker = np.array([0.98, 0.9, -0.9]) # 고전영화와 동떨어진 최신 영화이므로 약한매치로 표현되었다.

# 현대적인 SF 영화를 좋아하는 사용자는 다음과 같이 표현된다.
user1 = np.array([0.9, 0.8, -0.6])

In [5]:
# 위 영화와 유저 간의 유사성을 계산할 수 있다.

(last_skywalker * user1).sum(), (last_skywalker * user1)

# 벡터 곱 후 모두 더하는 연산을 점곱이라 한다.

(2.1420000000000003, array([0.882, 0.72 , 0.54 ]))

In [6]:
# <카사블랑카와 유저의 조합>

casablanca = np.array([-0.99, -0.3, 0.8])

(casablanca * user1).sum()

# 위와 같이 SF, Action, Classic 에 대한 요소의 값을 안다는 게 잠재요소다.

-1.611

# 잠재 요소 학습하기

SGD를 통해 학습할 수 있으며 파라미터는 임의의 랜덤값으로 초기화 한다.\
잠재요소의 갯수는 5개로 설정하였으며 이후 잠재요소의 설정방법을 다루게 된다.\

사용자와 영화에 잠재요소가 존재하며 이를 가로, 세로에 배치하여 점곱으로 점수를 넣는다.\
해당 점수는 실제 사용자의 평점과 비교하는데 이때 손실함수로는 MSE를 사용한다.\

학습을 통해 학습률을 조정하고 파라미터를 수정하면 손실이 점점 줄어들고 추천점수도 어느 정도 정확하게\
만들어진다. 만들어진 점수를 토대로 사용자에게 추천을 해준다.

In [7]:
movies = pd.read_csv(path/'u.item', delimiter = '|', encoding='latin-1',
                     usecols=(0,1), names = ('movie', 'title'), header = None)
movies.head()

# movies.iloc[241]

# movie             242
# title    Kolya (1996)
# Name: 241, dtype: object

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
ratings = ratings.merge(movies)
ratings.head() # 무비의 숫자에 맞게 타이틀이 merge 되었다.

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [9]:
# 협업 필터링에 특화된 CollabDataLoaders로 첫 번째를 사용자, 두 번째를 항목(items), 세 번째를 점수로 설정한다.
# 항목에 영화의 타이틀을 넣을 거고 item_name 인자에 값을 넣어주면 항목으로 설정한다

dls = CollabDataLoaders.from_df(ratings, item_name = 'title', bs = 64)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [10]:
# 파이토치로 교차표를 사용할 수가 없다.
# 영화와 사용자의 잠재요소를 행렬로 표현한다.

n_users = len(dls.classes['user']) # 944개
n_movies = len(dls.classes['title']) # 1665개
n_factors = 5


user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

# user_factors는 행 944개, 열 5개인 정규분포의 랜덤값이 만들어지고
# movie_factors는 행 1665개, 열 5개인 정규분포의 랜덤값이 만들어졌다.

In [11]:
# 유저와 특정 영화에 대한 계산을 할려면 색인을 해야 한다. 그러나 그건 딥러닝이 하는 게 아니라 행렬의 곱셈으로, one-hot encoding으로 색인을 대체하면 된다.

one_hot_3 = one_hot(3, n_users).float() # 인덱스 3인 부분에서만 1의 값을 가진다. one-hot encoding으로 나머지는 다 0이다.
# user_factors.t() 행렬을 바꿈

user_factors.t().shape, one_hot_3.shape
# (torch.Size([5, 944]), torch.Size([944]))

user_factors.t() @ one_hot_3

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

In [12]:
# 위 결과는 행렬의 3번째 색인에 해당되는 벡터와 같다

user_factors[3]

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

# 밑바닥부터 만드는 협업 필터링

In [13]:
# 파이썬은 객체지향 프로그래밍을 선호하고 그 중심에는 Class가 있다.
# 그 예로 우리는 DataLoaders, DataBlock 등을 사용해왔다.

class Example:
  def __init__(self, a):
    self.a = a
  def say(self, x):
    return f'Hello {self.a}, {x}'
  
ex = Example('Beom')
# ex.a  Beom

ex.say('nice to meet u')

'Hello Beom, nice to meet u'

In [14]:
# 파이토치 모듈을 만들기 위해선 Module을 상속해야 한다. 상위 클래스..

# 점곱의 클래스 만들기

class DotProduct(Module):
  def __init__(self, n_users, n_movies, n_factors):
    self.user_factors = Embedding(n_users, n_factors) # embedding 클래스를 상속받아서 사용한다.
    self.movie_factors = Embedding(n_movies, n_factors)
  
  def forward(self, x):
    users = self.user_factors(x[:,0]) # 첫 번째 열에는 사용자의 ID
    movies = self.movie_factors(x[:,1]) # 두 번째 열에는 영화 ID 
    return (users * movies).sum(dim=1)


In [15]:
x, y = dls.one_batch()
x.shape # 유저 아이디와 영화 ID를 나타내고 y는 그거에 따른 평점이다.
# Collab이 자동으로 설정했다.

torch.Size([64, 2])

In [16]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [17]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.344786,1.279100,00:07
1,1.093331,1.109981,00:16
2,0.958258,0.990199,00:07
3,0.814234,0.894916,00:07
4,0.780714,0.882022,00:09


In [18]:
class DotProduct(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
    self.user_factors = Embedding(n_users, n_factors) 
    self.movie_factors = Embedding(n_movies, n_factors)
    self.y_range = y_range # 예측값을 0과 5.5 사이로 강제했다. 5.5로 하면 더 나은 결과를 얻을 수 있어서
  
  def forward(self, x):
    users = self.user_factors(x[:,0]) 
    movies = self.movie_factors(x[:,1])
    return sigmoid_range((users * movies).sum(dim=1), *self.y_range)


model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3) # 합리적인 출발선을 그었다. 시작 오차가 크지 않아짐을 알 수 있다.

epoch,train_loss,valid_loss,time
0,0.986799,1.005294,00:15
1,0.878134,0.918898,00:12
2,0.675850,0.875467,00:13
3,0.483372,0.877939,00:15
4,0.378927,0.881887,00:13


In [19]:
# 어떤 사용자는 다른 사용자에 비해 영화에 대해 더 긍정적이거나 부정적일 수 있고,
# 어떤 영화는 다른 영화보다 더 좋거나 나쁠 수 있다는 사실이다

# 이는 가중치와 함께 편향이 있어야 한다는 것이다.

class DotProductBias(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
    self.user_factors = Embedding(n_users, n_factors) 
    self.user_bias = Embedding(n_users, 1)
    self.movie_factors = Embedding(n_movies, n_factors)
    self.movie_bias = Embedding(n_movies, 1)
    self.y_range = y_range # 예측값을 0과 5.5 사이로 강제했다. 5.5로 하면 더 나은 결과를 얻을 수 있어서
  
  def forward(self, x):
    users = self.user_factors(x[:,0]) 
    movies = self.movie_factors(x[:,1])
    res = ((users * movies).sum(dim=1, keepdim = True)) 
    res += self.user_bias(x[:,0]) + self.movie_bias(x[:, 1]) # 행렬 곱이 된 후 각 컬럼마다 편향을 더해준다.
    return sigmoid_range(res, *self.y_range) 



model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.938634,0.952516,00:14
1,0.846664,0.865633,00:14
2,0.608090,0.865127,00:15
3,0.413482,0.887318,00:08
4,0.286971,0.894876,00:08


In [20]:
# 가중치 감소

# 과적합이 일어날 때 데이터를 증강시킬 수 없다면 정규화 기법이 필요하다.
# 계수가 클 수록 손실함수에서 경사가 급격해지는 협곡이 나타난다는 것이다. 
# 변화가 매우 심하면 학습용 데이터의 모든 데이터에 적합하게 된다. = 과적합

# 가중치를 제한하는 것은 모델의 학습을 방해하지만 일반화를 더 잘 되게 만들어준다.

# wd 는 fit, fit_one_cycle에서 인자로 사용할 수 있다.

# 공부 필요!!


model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.2)

epoch,train_loss,valid_loss,time
0,0.942720,0.971832,00:08
1,0.923035,0.908290,00:08
2,0.855419,0.856756,00:08
3,0.744558,0.834173,00:08
4,0.682569,0.830827,00:09


## 나만의 임베딩 모듈 만들기

In [21]:
# 임베딩의 가중치 행렬은 임의로 초기화되어야 한다. 파라미터를 호출하기 위해 텐서를 추가하더라도 파라미터 그룹에 자동으로 포함되지 않는다.

class T(Module):
  def __init__(self): self.a = torch.ones(3)

L(T().parameters())

(#0) []

In [22]:
# nn.parameters로 wrap해야 한다.

class T(Module):
  def __init__(self): self.a = nn.Parameter(torch.ones(3))

L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [23]:
# 모든 파이토치 모듈은 학습 가능한 파라미터(SGD를 통해, ..) nn.Parameter를 사용하므로 명시적으로 사용할 필요는 없다.

class T(Module):
  def __init__(self): self.a = nn.Linear(1,3, bias=False)

t = T()
L(t.parameters()), type(t.a.weight)

((#1) [Parameter containing:
tensor([[-0.3292],
        [-0.8623],
        [ 0.0592]], requires_grad=True)],
 torch.nn.parameter.Parameter)

In [24]:
# 임의 초기화된 텐서를 파라미터로 생성할 수 있다.

def create_params(size):
  return nn.Parameter(torch.zeros(*size).normal_(0, 0.01)) # normal_은 해당 값을 정규분포에 의거한 값을 빈 값에 채운다는 것을 의미

In [25]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:,0]]
        movies = self.movie_factors[x[:,1]]
        res = (users*movies).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)

In [26]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.929254,0.953444,00:08
1,0.865246,0.878304,00:08
2,0.720294,0.838921,00:09
3,0.582796,0.829129,00:08
4,0.474043,0.829031,00:08


# 임베딩과 편향의 분석

발견한 파라미터 중 가장 분석이 쉬운 건 bias다.
낮은 편향 벡터는 모든 사람들에게 있어서 그닥 좋아하지 않은 영화(평점이 다 낮음)\
높은 편향 벡터는 모든 사람들에게 있어 취향 타지 않고 다 좋아하는 영화(평점 높음)

In [27]:
movie_bias = learn.model.movie_bias.squeeze()
idx = movie_bias.argsort()[:5] # 편향의 값이 오름차순으로 정렬한 뒤 인덱스 형식으로 반환되는데(영화의 len과 같음) index 5번째 값까지 값을 idx에 담는다.
[dls.classes['title'][i] for i in idx]

['Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Children of the Corn: The Gathering (1996)',
 'Mortal Kombat: Annihilation (1997)',
 'Amityville 3-D (1983)',
 'Beautician and the Beast, The (1997)']

In [28]:
# 편향이 높게 나온 영화들 

movie_bias = learn.model.movie_bias.squeeze()
idx = movie_bias.argsort(descending=True)[:5] # 편향의 값이 내림차순으로 역정렬
[dls.classes['title'][i] for i in idx]

['Titanic (1997)',
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'L.A. Confidential (1997)',
 "Schindler's List (1993)"]

In [29]:
# collab_learner 함수를 쓰면 앞에와 같이 정확한 구조로 협업 필터링 모델을 만들고 학습할 수 있다.

learn = collab_learner(dls, n_factors=50, y_range=(0,5.5))

learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.939463,0.954959,00:08
1,0.841215,0.876151,00:08
2,0.724404,0.832099,00:09
3,0.597228,0.816953,00:08
4,0.481373,0.817286,00:08


In [30]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [31]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['L.A. Confidential (1997)',
 'Titanic (1997)',
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Rear Window (1954)']

In [47]:
# 피타고라스의 정리를 통해 두 벡터 간의 거리를 파악할 수 있다.
# 영화 간의 유사성은 대상 영화들을 좋아하는 사용자간의 유사성으로 정의할 수 있따.

# <양들의 침묵과 유사한 영화찾기>

movie_factors = learn.model.i_weight.weight
# movie_factors.shape [1664, 50]
idx = dls.classes['title'].o2i['Silence of the Lambs, The (1991)']
# idx >> 1330 int
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]
# idx >> tensor([1330,  149, 1606,  ..., 1246, 1346,  935], device='cuda:0') descending=True니까 distance에서 가장 값이 큰 게 가장 비슷한 것
# 근데 distance면 거리인데 이게 값이 작아야 좋은 거 아닌가? 왜 이렇지?

# 결과보니 학습이 잘못된 것 같기도 하고?

'Before the Rain (Pred dozhdot) (1994)'

# 초기의 협업 필터링 모델 구축하기

첫 사용자 같은 경우 어떠한 데이터도 보유하고 있지 않기 때문에 추천하기가 어렵다 \

평균 취향을 가진 사용자를 선택하는 것도 대안이지만 \

초기에 어떤 질문을 통해 응답을 독립변수로 임베딩 벡터를 종속변수로 설정해줄 수 있다. \

특이 값으로 몰리는 걸 막기위해 사람을 피드백 루프에 투입 시킨다.

# 협업 필터링을 위한 딥 러닝

임베딩 결과를 활성에 연결하는 것이다. 그러면 행렬이 만들어진다.

점곱을 계산하지 않고 임베딩 행렬을 이어붙여 임베딩 행렬의 크기가 다를 수 있다.

임베딩 행렬에 권장되는 크기를 반환하는 get_emb_sz 함수를 제공한다.

In [48]:
embs = get_emb_sz(dls)
embs # 잠재요소가 각각 74, 102개씩

[(944, 74), (1665, 102)]

In [49]:
from torch.nn.modules.activation import ReLU
class CollabNN(Module):
  def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
    self.user_factors = Embedding(*user_sz)
    self.item_factors = Embedding(*item_sz)
    self.layers = nn.Sequential(
        nn.Linear(user_sz[1] + item_sz[1], n_act), # input size = user_sz[1] + item_sz[1] , output size = 100
        nn.ReLU(),  # 비선형함수
        nn.Linear(n_act, 1) # input 100, output 1
    )
    self.y_range = y_range

  def forward(self, x):
    embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
    x = self.layers(torch.cat(embs, dim=1))
    return sigmoid_range(x, *self.y_range)

In [55]:
# 모델 생성

model = CollabNN(*embs)

In [57]:
# 학습 
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.945274,0.954863,00:12
1,0.915954,0.899382,00:08
2,0.850915,0.882266,00:08
3,0.814545,0.876799,00:08
4,0.765348,0.877984,00:08


In [58]:
# 다음과 같은 클래스를 사용하기 위해서는 collab_learner 함수 호출 시 use_nn=True를 인잣값으로 넘겨주면(내부적으로 get_emb_sz 호출) 된다.

# 계층을 더 쌓기 위해서는 layers 파라미터에 리스트로 은닉 계층의 크기를 정하면 된다.

learn = collab_learner(dls, use_nn=True, y_range=(0,5.5), layers=[200,100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.008797,0.988152,00:19
1,0.881503,0.926622,00:19
2,0.893006,0.902641,00:12
3,0.831988,0.877496,00:11
4,0.815085,0.874785,00:11


In [59]:
# learn.model은 EmbeddingNN 유형의 객체

@delegates(TabularModel)
class EmbeddingNN(TabularModel):
  def __init__(self, emb_szs, layers, **kwargs):
    super().__init__(emb_szs, layers=layers, n_cont=0, out_sz=1, **kwargs)